<a href="https://colab.research.google.com/github/niapollard/Benchmarking_QC/blob/main/051324.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q jarvis-tools qiskit==0.43.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
!pip install qiskit_nature==0.6.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 8.7 MB/s eta 0:00:00


In [3]:
!pip install pyscf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 MB 12.7 MB/s eta 0:00:00


In [4]:
!pip install qiskit-aer==0.12.0

In [5]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.drivers.second_quantization import MethodType
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.transformers.second_quantization.electronic.active_space_transformer import ActiveSpaceTransformer
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
import itertools, functools
from qiskit.opflow import I, X, Y, Z
from jarvis.db.figshare import get_wann_electron, get_wann_phonon, get_hk_tb
from jarvis.core.circuits import QuantumCircuitLibrary

In [6]:
molecule = Molecule(
    geometry=[['Al',[14.46097199, -11.1250000, 0.00000000]],
              ['Al',[17.61662800, -11.1250000, 0.00000000]]],
    charge=0,
    multiplicity=1)

In [7]:
print(molecule)

Molecule:
	Multiplicity: 1
	Charge: 0
	Unit: Angstrom
	Geometry:
		Al	[14.46097199, -11.125, 0.0]
		Al	[17.616628, -11.125, 0.0]


In [7]:
def get_qubit_op(
    molecule,
    basis='sto3g',
    functional='lda',
    method= MethodType.RKS,
    driver_type= ElectronicStructureDriverType.PYSCF,
    converter= JordanWignerMapper()):

  driver=ElectronicStructureMoleculeDriver(molecule, basis, method,driver_type)
  properties = driver.run()
  problem = ElectronicStructureProblem(driver)

  second_q_ops = problem.second_q_ops()
  second_q_ops = list(second_q_ops.values()) #list of second quantized operators
  hamiltonian=second_q_ops[0]

  numpy_solver = NumPyMinimumEigensolver()
  tmp = properties.get_property('ParticleNumber')
  alpha_occ=tmp.occupation_alpha
  beta_occ=tmp.occupation_beta

  mo_considered=3
  #active and inactive space has to be even, non-magnetic

  first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

  orb_act = np.arange(first_index-mo_considered,first_index)+1

  transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
  problem_reduced = ElectronicStructureProblem(driver, [transformer])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  second_q_ops_reduced = list(second_q_ops_reduced.values())
  hamiltonian_reduced = second_q_ops_reduced[0]
  mapper = JordanWignerMapper()
  converter = QubitConverter(mapper=mapper)
  qubit_op = converter.convert(hamiltonian_reduced)
  return qubit_op, converter

In [37]:
# qubit_op = get_qubit_op(molecule,
#                            basis='sto3g',
#                            functional='lda',
#                            method= MethodType.RKS,
#                            driver_type= ElectronicStructureDriverType.PYSCF,
#                            converter= JordanWignerMapper())

# print(qubit_op)

<ipython-input-36-736ccb2176dc>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


(PauliSumOp(SparsePauliOp(['IIIIII', 'IIIIIZ', 'IIIIZI', 'IIIZII', 'IIZIII', 'IZIIII', 'ZIIIII'],
              coeffs=[ 3. +0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j]), coeff=1.0), <qiskit_nature.converters.second_quantization.qubit_converter.QubitConverter object at 0x7e4ebca133d0>)


In [10]:
def get_energy(optimizer='',device='',qubit_op='',seed=''):
  seed = 42
  counts = []
  values = []


  def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

  algorithm_globals.random_seed = seed
  print('device',device)
  print('seed',seed)

  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
  n_qubits = qubit_op.num_qubits
  ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=1).circuit6()
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)
  result = vqe.compute_minimum_eigenvalue(operator=qubit_op)


  eigenvalue = result.eigenvalue
  return eigenvalue, vqe, qi


# n_qubits = qubit_op.num_qubits
# ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=1).circuit6()
# qubit_op = get_qubit_op(molecule,
#                            basis='sto3g',
#                            functional='lda',
#                            method= MethodType.RKS,
#                            driver_type= ElectronicStructureDriverType.PYSCF,
#                            converter= JordanWignerMapper())
#optimizer=SLSQP(maxiter=1000)
#device= Aer.get_backend('statevector_simulator')
# en=get_energy(optimizer=optimizer,device=device,qubit_op=qubit_op,seed=42,ansatz=ansatz)

elements=['Al']
basis=['sto3g']
method=[MethodType.RKS]
#driver_type=[]
optimizer=[SLSQP(maxiter=1000)]
converter=[JordanWignerMapper()]
functions=['lda']
devices=[Aer.get_backend('statevector_simulator')]
mem={}
for i in elements:
  for j in basis:
    for k in method:
      #for l in driver_type:
        for m in optimizer:
          for n in converter:
            for o in functions:
              for p in devices:
                 print(i,j,k,m,n,o,p)
                #  pos=make_diatomic_system(element=i,bond_distance=1.2)
                 qubit_op,converter=get_qubit_op(molecule=molecule,basis=j,functional=o,method=k,converter=n)
                 en,vqe,qi=get_energy(optimizer=m,device=p,qubit_op=qubit_op,seed=42)
                 print(en)
                 mem[i,j,k,m,n,o,p]=en
print(mem)

Al sto3g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7a02475993f0> <qiskit_nature.mappers.second_quantization.jordan_wigner_mapper.JordanWignerMapper object at 0x7a024759b310> lda statevector_simulator


<ipython-input-7-736ccb2176dc>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-10-c3a868ed32e5>:15: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-10-c3a868ed32e5>:18: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
{('Al', 'sto3g', <MethodType.RKS: 'rks'>, <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7a02475993f0>, <qiskit_nature.mappers.second_quantization.jordan_wigner_mapper.JordanWignerMapper object at 0x7a024759b310>, 'lda', StatevectorSimulator('statevector_simulator')): (8.5066017987e-08+0j)}


In [27]:
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver

solver =GroundStateEigensolver(converter, vqe)
result=solver.solve(problem_reduced)
print(result)

# qubit_op = get_qubit_op(molecule,
#                            basis='sto3g',
#                            functional='lda',
#                            method= MethodType.RKS,
#                            driver_type= ElectronicStructureDriverType.PYSCF,
#                            converter= JordanWignerMapper())

NameError: name 'vqe' is not defined

In [13]:
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver

solver =GroundStateEigensolver(converter, vqe)
result=solver.solve(problem_reduced)
print(result)

<ipython-input-13-fb24484ecdc7>:1: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
<ipython-input-13-fb24484ecdc7>:1: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver


NameError: name 'vqe' is not defined

In [ ]:
QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)

In [ ]:
)#print(result)
print('VQE eig.', eigenvalue)

In [ ]:
hamiltonian,hamiltonian_reduced = get_qubit_op(molecule,
                           basis='sto3g',
                           functional='lda',
                           method= MethodType.RKS,
                           driver_type= ElectronicStructureDriverType.PYSCF,
                           converter= JordanWignerMapper())

print(hamiltonian_reduced)

<ipython-input-14-8d355a39e2b3>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


Fermionic Operator
register length=6, number terms=6
  (1+0j) * ( +_0 -_0 )
+ (1+0j) * ( +_1 -_1 )
+ (1+0j) * ( +_2 -_2 )
+ (1+0j) * ( +_3 -_3 )
+ (1+0j) * ( +_4 -_4 )
+ (1+0j) * ( +_5 -_5 )


In [ ]:
# Call the function to get the Hamiltonian
hamiltonian = get_qubit_op(molecule,
                           basis='sto3g',
                           functional='lda',
                           method= MethodType.RKS,
                           driver_type= ElectronicStructureDriverType.PYSCF,
                           converter= JordanWignerMapper())

# Print the Hamiltonian
print(hamiltonian)

Fermionic Operator
register length=36, number terms=36
  (1+0j) * ( +_0 -_0 )
+ (1+0j) * ( +_1 -_1 )
+ (1+0j) * ( +_2 -_2 )
+ (1+0j) * ( +_3 -_3 )
+ (1+0j) * ( +_4 -_4 )
+ (1+0j) * ( +_5 -_5 )
+ (1+0j) * ( +_6 -_6 )
+ (1+0j) * ( +_7 -_7 )
+ (1+0j) * ( ...


In [ ]:
numpy_solver = NumPyMinimumEigensolver()
  tmp = properties.get_property('ParticleNumber')
  alpha_occ=tmp.occupation_alpha
  beta_occ=tmp.occupation_beta

  mo_considered=3
  #active and inactive space has to be even, non-magnetic

  first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

  orb_act = np.arange(first_index-mo_considered,first_index)+1

  transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
  problem_reduced = ElectronicStructureProblem(driver, [transformer])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  second_q_ops_reduced = list(second_q_ops_reduced.values())
  hamiltonian_reduced = second_q_ops_reduced[0]

  qubit_op = converter.convert(hamiltonian_reduced)
  return qubit_op

In [ ]:
seed = 42
counts = []
values = []


def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)
n_qubits = qubit_op.num_qubits
ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=1).circuit6()
slsqp = SLSQP(maxiter=1000)
vqe = VQE(ansatz, optimizer=slsqp, quantum_instance=qi,callback=store_intermediate_result)

result = vqe.compute_minimum_eigenvalue(operator=qubit_op)


eigenvalue = result.eigenvalue
#print(result)
print('VQE eig.', eigenvalue)

<ipython-input-31-bdb382c05e81>:11: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)


NameError: name 'qubit_op' is not defined